In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import  svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import time
import os

tqdm.pandas()
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

warnings.filterwarnings("ignore")

plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (20,10)

In [2]:
# read all of csvs
for name in os.listdir("foursquare-location-matching/"):
    if '.csv' in name:
        locals()[name[:-4]] = pd.read_csv("foursquare-location-matching/" + name)
        print(name)  
      

pairs.csv
sample_submission.csv
test.csv
train.csv


In [3]:
def convert_str_columns_to_lower_case(df):
    for column, column_type in pd.DataFrame(df.dtypes.astype(str)).reset_index().values:
        if column_type == "object":
            df[column] = df[column].str.lower()
    return df

def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km
    
# Usage
lon1 = -103.548851
lat1 = 32.0004311
lon2 = -103.6041946
lat2 = 33.374939  
print(haversine_np(lat1, lon1, lat2, lon2))  

36.37807003595054


In [4]:
#inplace
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Determines return value units.
    return c * r

# Usage
lon1 = -103.548851
lat1 = 32.0004311
lon2 = -103.6041946
lat2 = 33.374939

print(haversine(lat1, lon1, lat2, lon2))

36.40092417135871


In [19]:
raw_pair_df = pairs
raw_test_df = test

In [20]:
raw_pair_df = raw_pair_df.pipe(convert_str_columns_to_lower_case)
raw_test_df = raw_test_df.pipe(convert_str_columns_to_lower_case)
raw_pair_df

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,phone_1,categories_1,id_2,name_2,latitude_2,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,e_000001272c6c5d,café stad oudenaarde,50.860,3.634,abdijstraat,nederename,oost-vlaanderen,9700,be,NaN,NaN,bars,e_da7fa3963561f8,café oudenaarde,50.869,3.635,NaN,NaN,NaN,NaN,be,NaN,NaN,bars,True
1,e_000008a8ba4f48,turkcell,37.845,27.844,adnan menderes bulvarı,NaN,NaN,NaN,tr,NaN,NaN,mobile phone shops,e_3f12df0b045062,turkcell,37.838,27.846,batı aydın,aydın,NaN,67500,tr,NaN,NaN,electronics stores,False
2,e_000023d8f4be44,island spa,14.519,121.019,"5th flr, newport mall, resorts world manila",pasay city,metro manila,NaN,ph,NaN,NaN,spas,e_12453effe251db,island spa theater,14.518,121.020,NaN,NaN,NaN,NaN,ph,NaN,NaN,spas,True
3,e_00007dcd2bb53f,togo's sandwiches,38.258,-122.065,"1380 holiday ln., ste. b",fairfield,ca,94534,us,https://locations.togos.com/ll/us/ca/fairfield...,7074394747,sandwich places,e_f131dcb7f07be9,togo's,38.258,-122.065,NaN,fairfield,ca,NaN,us,NaN,NaN,sandwich places,True
4,e_0000c362229d93,coffee cat,7.082,125.610,f. torres st.,davao city,davao region,8000,ph,NaN,NaN,"cafés, coffee shops",e_d52c3c4d53831c,coffeecat,7.076,125.611,e. jacinto extension,davao city,davao region,8000,ph,NaN,NaN,"coffee shops, cafés, dessert shops",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578902,e_ffff80f94b2fee,blk 332 hougang avenue 5,1.368,103.898,332 hougang ave 5,singapore,singapore,530332,sg,NaN,NaN,residential buildings (apartments / condos),e_75feaa1e0321cc,blk332 carpark,1.368,103.897,hougang ave 5,NaN,NaN,NaN,sg,NaN,NaN,NaN,True
578903,e_ffff8cc1b92ab3,fawsley hall,52.206,-1.179,fawsley nr daventry,daventry,northants,nn11 3ba,gb,NaN,NaN,hotels,e_ad351ad024c4dd,the spa at fawsley hall,52.207,-1.180,NaN,NaN,NaN,NaN,gb,NaN,NaN,gyms or fitness centers,False
578904,e_ffff989ae206f8,cricket wireless authorized retailer,36.131,-86.716,825c murfreesboro pike,nashville,tn,37217,us,https://www.cricketwireless.com/stores/ll/us/t...,+16156799940,mobile phone shops,e_a5bc397a4eaeff,cricket wireless authorized retailer,36.135,-86.724,825c murfreesboro pike,nashville,tn,37217,us,https://www.cricketwireless.com,+16156799940,mobile phone shops,True
578905,e_ffffca745329ed,焼肉 和家,35.711,139.775,上野6-13-6,taitō,東京都,110-0005,jp,NaN,338362989,bbq joints,e_04988888cfff60,nagomiya,35.710,139.775,NaN,NaN,NaN,NaN,jp,NaN,NaN,bbq joints,True


## KDTree

In [22]:
%%time
tree = KDTree(raw_test_df[["latitude","longitude"]].values)
tree

Wall time: 994 µs


In [23]:
%%time
X = haversine_np(raw_pair_df['longitude_1'],
                  raw_pair_df['latitude_1'],
                  raw_pair_df['longitude_2'],
                  raw_pair_df['latitude_2'])
y = raw_pair_df.match.values

Wall time: 73.7 ms


In [42]:
%%time

## change
X = haversine(raw_pair_df['longitude_1'],
                  raw_pair_df['latitude_1'],
                  raw_pair_df['longitude_2'],
                  raw_pair_df['latitude_2'])
y = raw_pair_df.match.values

Wall time: 85.7 ms


In [15]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

print(classification_report(y_test, model.predict(X_test).astype(int)))

              precision    recall  f1-score   support

       False       0.35      0.34      0.35     35901
        True       0.71      0.72      0.71     79881

    accuracy                           0.60    115782
   macro avg       0.53      0.53      0.53    115782
weighted avg       0.60      0.60      0.60    115782



## SVC

In [18]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.2, random_state=42)

svc = svm.SVC()
svc.fit(X_train, y_train)

print(classification_report(y_test, svc.predict(X_test).astype(int)))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00     35901
        True       0.69      1.00      0.82     79881

    accuracy                           0.69    115782
   macro avg       0.34      0.50      0.41    115782
weighted avg       0.48      0.69      0.56    115782



## Logistic 

In [43]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.2, random_state=42)
softmax_reg = LogisticRegression(multi_class='multinomial', C=1)
softmax_reg.fit(X_train,y_train)


print(classification_report(y_test, softmax_reg.predict(X_test).astype(int)))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00     35901
        True       0.69      1.00      0.82     79881

    accuracy                           0.69    115782
   macro avg       0.34      0.50      0.41    115782
weighted avg       0.48      0.69      0.56    115782



In [44]:
softmax_reg.score(X_train,y_train)

0.6885937921727395

## SGD_classifier

In [37]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.2, random_state=42)
sgd_clf = SGDClassifier(loss='log', penalty='l2', alpha=0.0001)
sgd_clf.fit(X_train,y_train)

print(classification_report(y_test, sgd_clf.predict(X_test).astype(int)))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00     35901
        True       0.69      1.00      0.82     79881

    accuracy                           0.69    115782
   macro avg       0.34      0.50      0.41    115782
weighted avg       0.48      0.69      0.56    115782



In [38]:
sgd_clf.score(X_train,y_train)

0.6885937921727395

## DecisionTree

In [28]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.2, random_state=42)

# 建立分類器
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test).astype(int)))

              precision    recall  f1-score   support

       False       0.35      0.37      0.36     35901
        True       0.71      0.69      0.70     79881

    accuracy                           0.59    115782
   macro avg       0.53      0.53      0.53    115782
weighted avg       0.60      0.59      0.59    115782



In [ ]:
plt.figure(figsize=(5, 5))
tree.plot_tree(clf, feature_names=['x1', 'x2'], 
               class_names=['lon','lat'], rounded=True, filled=True)
plt.savefig("tree.pdf")
plt.show()

## 貝氏分類器

In [25]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.2, random_state=42)

BS = GaussianNB()
BS.fit(X_train, y_train)
BS.predict(X_test)

print(classification_report(y_test, BS.predict(X_test).astype(int)))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00     35901
        True       0.69      1.00      0.82     79881

    accuracy                           0.69    115782
   macro avg       0.34      0.50      0.41    115782
weighted avg       0.48      0.69      0.56    115782



In [47]:
submission_df_list = []
base_raw_df = raw_test_df[["id","latitude","longitude"]]
base_raw_df["dummy"] = 1

for row_id, row in tqdm(base_raw_df[["id","latitude","longitude"]].iterrows(), total=base_raw_df.shape[0]):    
    row_df = pd.DataFrame(row).T.astype({"latitude":"float","longitude":"float"})
    row_df["dummy"] = 1
    test_df = row_df.merge( 
           base_raw_df.iloc[ tree.query([row["latitude"],row["longitude"]], min(base_raw_df.shape[0],40))[1]],
           on = "dummy", suffixes=["_1","_2"])
    
    features = haversine_np(test_df['longitude_1'],
                  test_df['latitude_1'],
                  test_df['longitude_2'],
                  test_df['latitude_2'])
    
    ids_ = test_df[["id_1", "id_2"]].values
    prediction = clf.predict(pd.DataFrame(features)).astype(int)
    
    df = pd.DataFrame(ids_)
    df["prediction"] = prediction
    submission_ = (df
                    .query("prediction == 1")
                    .groupby(0, as_index=False)
                    .agg({1:" ".join})
                    .rename(columns={0:"id", 1:"matches"})
                    )
    submission_df_list.append(submission_)

  0%|          | 0/5 [00:00<?, ?it/s]

In [48]:
submission = pd.concat(submission_df_list)

In [ ]:
submission_base_file = pd.read_csv("D:/master/統計學習/專題/foursquare-location-matching/sample_submission.csv")
submission_data = submission_base_file.merge(submission, how="left",
                                             on=["id"]).drop("matches_x", axis=1).rename(columns={"matches_y":"matches"})
submission_data.matches = submission_data.matches.fillna(submission_data.id)

In [ ]:
submission.to_csv("submission.csv", index=False)